# Production-Ready Agent Workflow with Flotorch and LangGraph

This notebook demonstrates a comprehensive, production-style workflow. It integrates all the advanced features into a single, robust application:
- **Remote Agent Configuration**: Uses `FlotorchLangGraphAgent` to manage agent setup with custom tools.
- **Custom Local Tools**: Defines and attaches `multiply` and `addition` tools at runtime.
- **Dual Memory System**: Implements both session-based (short-term) memory and long-term external memory.

### Prerequisites
Configure agent, memory provider, and API key in the Flotorch console (https://console.flotorch.cloud/).

### Viewing Logs
Logs can be viewed in the logs tab in the Flotorch console (https://console.flotorch.cloud/).

## 1. Setup and Imports

The following cells install dependencies, set API credentials, and import all necessary components for handling tools, memory, and remote agent management.

In [ ]:
%pip install flotorch[langgraph]

In [ ]:
FLOTORCH_API_KEY = "<YOUR FLOTORCH_API_KEY >"
FLOTORCH_BASE_URL = "<YOUR FLOTORCH_BASE_URL>"  
FLOTORCH_MODEL_ID = "<YOUR FLOTORCH_MODEL_ID>"
AGENT_NAME = "<YOUR AGENT_NAME>"  #eg : adk
PROVIDER_NAME = "<YOUR PROVIDER_NAME>" #eg : memo-provider
USER_ID = "YOUR USER_ID" #eg : flotorch_user1500
APP_ID = "YOUR APP_ID"   #eg : flotorch_app1500

In [ ]:
import nest_asyncio
nest_asyncio.apply()


In [ ]:
from langchain.agents import tool
from flotorch.langgraph.agent import FlotorchLangGraphAgent
from flotorch.langgraph.sessions import FlotorchLanggraphSession
from flotorch.langgraph.memory import FlotorchStore

print("✔ Imported necessary libraries successfully")

## 2. Defining a Custom Tool

To extend the agent's capabilities, we define a single custom tool using the `@tool` decorator from LangChain. The `multiply` tool takes a comma-separated string of two integers (e.g., 'x,y') and returns their product. The tool's docstring provides a clear description, enabling the agent to understand its purpose and usage.

In [ ]:
@tool
def multiply(numbers: str) -> int:
    """
    Multiply two integers provided as a comma-separated string.

    Args:
        numbers (str): Two integers in the format 'x,y' (e.g., '3,4')

    Returns:
        int: The product of the two integers
    """
    print("Multiplying... custom tool")
    a, b = map(int, numbers.split(","))
    return a * b


print("✔ Custom multiplication tool defined successfully.")

## 3. Defining a Custom Tool

To extend the agent's capabilities, we define a custom tool using the `@tool` decorator from LangChain. The `addition` tool takes a comma-separated string of two integers (e.g., 'x,y') and returns their sum. The tool's docstring provides a clear description, enabling the agent to understand its purpose and usage.

In [ ]:
@tool
def addition(numbers: str) -> int:
    """
    Addition of two integers provided as a comma-separated string.

    Args:
        numbers (str): Two integers in the format 'x,y' (e.g., '3,4')

    Returns:
        int: The Addition of the two integers
    """
    print("Adding... custom tool")
    a, b = map(int, numbers.split(","))
    return a + b

tools = [multiply,addition]

print("✔ Custom Addition tool defined successfully.")

## 4. Session Storage Setup

We configure the `FlotorchLanggraphSession` as the checkpointer for LangGraph, enabling persistent storage of conversation history within a session. The session is uniquely identified by `app_name`, `user_id`, and `thread_id`.

In [ ]:
checkpointer = FlotorchLanggraphSession(
    api_key=FLOTORCH_API_KEY,
    base_url=FLOTORCH_BASE_URL,
    app_name=APP_ID,
    user_id=USER_ID
)

config = {"configurable": {"thread_id": "flotorch-test-thread-1500"}}

print(f"✔ Session storage initialized with thread ID: {config['configurable']['thread_id']}")

## 5. Memory Configuration

- **External (Long-Term) Memory**: Uses `FlotorchLangGraphMemory` to store and retrieve key information across different sessions, acting as the agent's persistent knowledge base.

In [ ]:
memory = FlotorchStore(
    api_key=FLOTORCH_API_KEY,
    base_url=FLOTORCH_BASE_URL,
    provider_name=PROVIDER_NAME,
    userId=USER_ID,
    appId=APP_ID
)

print("✔ Initialized session-based and long-term memory")

 A custom `search_memories` function enhances the agent's ability to retrieve relevant past information.

In [ ]:
def search_memories(user_message):
    """Search for relevant memories based on user message."""
    try:
        items = memory.search(
            ("user_123", "memories"),
            query=user_message,
            limit=3
        )
        memories = "\n".join(item.value["text"] for item in items)
        return f"## Relevant Memories\n{memories}" if memories else ""
    except Exception as e:
        print(f"Memory search error: {e}")
        return ""


In [ ]:
def store_interaction_in_memory(user_query, assistant_reply):

    memory.put(
        (APP_ID, USER_ID, "conversations"), "1", 
        {"text": f"User: {user_query}"}
    )

    memory.put(
        (APP_ID, USER_ID, "conversations") ,"2", 
        {"text": f"Assistant: {assistant_reply}"}
    )
    
def enhance_query_with_memory(user_query):

    relevant_memories = search_memories(user_query)
    
    if relevant_memories:
        print(f"Found relevant memories: {relevant_memories}")
        return f"{user_query}\n\n{relevant_memories}"
    else:
        return user_query

print("✔ Improved memory functions defined")


## 6. Agent Initialization with Custom Tools

We initialize the `FlotorchLangGraphAgent` client, loading our agent's configuration and integrating the custom `multiple` and `addition` tools, along with the dual-memory system.

In [ ]:
client = FlotorchLangGraphAgent(
    agent_name =AGENT_NAME,
    api_key=FLOTORCH_API_KEY,
    base_url=FLOTORCH_BASE_URL,
    custom_tools = tools,
    checkpointer=checkpointer,
    store=memory
)

agent = client.get_agent()

print("✔ Initialized FlotorchLangGraphAgent with custom tools and memory")

## 6. Interactive Demonstration

Engage with the agent in this interactive loop to test its full range of capabilities. Try asking it to perform arithmetic (e.g., 'addition(3, 4)' or 'multiply(5, 6)') and recall information from both the current session (via `FlotorchLanggraphSession`) and long-term memory (via `FlotorchLangGraphMemory`, e.g., 'What do I love?' or 'What is my profession?'). Type 'exit' to end the session.

In [ ]:
while True:
    user_query = input("user: ")
    if user_query.lower().strip() == "exit":
        break
    
    enhanced_message = enhance_query_with_memory(user_query)
    
    response = agent.invoke({"messages": enhanced_message}, config)
    bot_reply = response["messages"][-1].content
    print(f"Assistant: {bot_reply}")

    store_interaction_in_memory(user_query, bot_reply)

print("✔ Interactive session ended.")


## Summary

This notebook served as a **capstone project**, successfully integrating all previously demonstrated concepts into a single, comprehensive, and production-ready workflow using LangGraph and Flotorch. We built an advanced AI agent that combines the power of **remote configuration**, **local custom tools**, and a **full dual-memory system**.

### Key Achievements

- **Hybrid Configuration**  
  Initialized an agent using `FlotorchLangGraphAgent` to load its base configuration, then augmented it at runtime with locally defined custom tools (`add_numbers` and `multiply_numbers`).

- **Full Memory Stack**  
  Equipped the agent with both `FlotorchLanggraphSession` for conversational flow and `FlotorchLangGraphMemory` for long-term knowledge persistence.

- **End-to-End Demonstration**  
  Showcased the agent's ability to seamlessly switch between using its custom tools for specific tasks and accessing its memory to provide contextually rich, intelligent responses.

